In [11]:
#This is the cell where I import everything used in this project. The chromedriver is in the folder. 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
#List of URL home pages. These were found by a manual search. 
exchange_URL = 'https://en.wikipedia.org/w/index.php?title=List_of_stock_exchanges&action=history'
weather_URL = 'https://www.wunderground.com/history'

driver = webdriver.Chrome('./chromedriver')
complete_list = []  #This list would contain all the data we would get from scraping the wikipedia page. 
driver.get(exchange_URL)


#This link changes the amount of links per page from 50 to 500. This is great as it covers our whole range. 
fivehundred_link = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[4]/a[7]')
fivehundred_link.click()


  #The chromedriver is selected and we are going to the stock exchange URL first. 


#Here is where I set the parameters 
edit_list = []
current_year = ''
current_month = ''
amount_of_ranks = 16
list_of_month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']



full_edit_list = driver.find_element_by_xpath('//*[@id="pagehistory"]') ##This is the block which contains all the links to the edited pages
edit_list = full_edit_list.find_elements_by_xpath("*") #This list is all the children of the full_edit_lsit
completed_months_and_years = [] # A list of  two-entried tuples. It marks all the months and years which have already been scraped. 
change_of_format = False #At some point, there is a change in format in the wikipedia page. When this turns true, the code is thereby altered. 
for link in edit_list:
    month_and_year = []
    written_date = link.find_element_by_class_name('mw-changeslist-date') # The text here gives us the date
    written_date_link = written_date.get_attribute('href')
    written_date_text = written_date.text
    if written_date_text == '07:40, 24 January 2021': #This link is broken. It must be skipped
        continue
    #The following piece of code concerns with deciphering the link's date


    current_year = written_date_text[-4:]    #The last four characters of the string always indicates its year.
    for month in list_of_month_names: #This goes through the actual months of the year and checks whether it is in the string or not. 
        if month in written_date_text:
            current_month = month #If the month is in the string, then the edit was made on that month of the year.
    month_and_year = (current_year, current_month) #The first entry of this list is the month of the edit and the second entry is the year.


    if written_date_text == '02:17, 22 April 2021': # This is how far we are going back as bugs start arising further back in the past. 
        change_of_format = True
    if month_and_year not in completed_months_and_years: #If the month has not been scrapped yet...This is to not have multiple datasets per month.
        this_months_dictonary = {}
        months_rank_list = []
        months_exchange_list = []
        months_location_list = []
        months_market_cap_list = []
        months_trade_volume_list = []
        second_driver = webdriver.Chrome('./chromedriver') # It makes it easier if we use a second driver to open links to a new window. 
        second_driver.get(written_date_link)

        #The code from here to the for loop gets the table as a list of rows. This makes it easier to scrape from 
        table_header = second_driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[1]')
        table_header_children =table_header.find_elements_by_xpath("*")
        current_column = 0 
        if change_of_format == False:
            page_table = second_driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[2]/tbody')
        else:
            page_table = second_driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody')
        list_of_rows = page_table.find_elements_by_xpath("*")
        for row in list_of_rows:    #For every row in the table
            list_of_row_elements = row.find_elements_by_xpath("*")
            k = 1   #This k indicates which column we are concerned with
            for row_elements in list_of_row_elements:
                if len(list_of_row_elements) == 16:  #This is the correct amount of elements in a row. Any other number than this makes it difficult to scrape from. 
                    if k == 1:  #This column corresponds to the rank of the exchange. 
                        months_rank_list.append(row_elements.text)

                    if k ==3: #This column corresponds to the name of the exchange.
                        months_exchange_list.append(row_elements.text)

                    if k == 6:
                        split_locations = row_elements.text.split('\n')
                        months_location_list.append(split_locations)

                    if k == 7:  #This column corresponds to the market's market cap. 
                        months_market_cap_list.append(row_elements.text)

                    if k ==8: # This column corresponds to the market's trade volume
                        months_trade_volume_list.append(row_elements.text)
                k += 1 #  Move to the next column 
                
                    
        #Here we are adding the information we have scrapped to this months dictionary. 
        this_months_dictonary['Rank'] = months_rank_list
        this_months_dictonary['Name of exchange'] = months_exchange_list
        this_months_dictonary['Locations'] = months_location_list
        this_months_dictonary['Market Cap'] = months_market_cap_list
        this_months_dictonary['Trade Voume'] = months_trade_volume_list
        complete_list.append([month_and_year, this_months_dictonary])                      
        second_driver.quit()
        completed_months_and_years.append(month_and_year)
    
    if written_date_text == '16:08, 6 August 2020': #The code gets completely different after this point. 
        break




In [16]:
#A rudimentary file which shall be throughighly cleaned later. 
df = pd.DataFrame(complete_list)
df.to_csv('exchanges_list.csv', index = False, sep= ';')

In [24]:
#Imbetween Part 
# First we have to get all the locations as a list without duplicates. In the future we are going to find the location codes of all these places. We looking everywhere where 
# the locations may be 
list_of_locations = []
for all_enteries in complete_list:
    for exchanges in all_enteries[1]['Locations']: #The zero entry would otherwise take us to the date. 
        for exchange_locations in exchanges:
            if exchange_locations not in list_of_locations: 
                list_of_locations.append(exchange_locations)
#print(complete_dictionary[0])

['New York City', 'Hong Kong', 'Shanghai', 'Tokyo', 'Osaka', 'Amsterdam', 'Brussels', 'Dublin', 'Lisbon', 'Milan', 'Oslo', 'Paris', 'Shenzhen', 'London', 'Mumbai', 'Toronto', 'Riyadh', 'Frankfurt', 'Seoul', 'Busan', 'Zurich', 'Taipei', 'Sydney', 'Johannesburg', 'São Paulo', 'Madrid', 'Singapore', 'Moscow', 'Bangkok', 'Mexico City', 'Jakarta', 'Market place']


In [ ]:
#Next part: Correlating each location to a code to search for. This cell is about finding the URL's. The next cell gets the code from the URL. 
import time  #We need to do a bit of waiting to make sure everything loads properly on the page we are using. 
driver = webdriver.Chrome('./chromedriver')
driver.get(weather_URL) # This is the page in the weather website where you can input the location and date and look for infomation. 
location_urls = []
for location in list_of_locations:
    time.sleep(4)
    search_bar = driver.find_element_by_name('historySearch')   #This is the search bar
    search_bar.click()
    if location == 'Hong Kong':
        location = 'Kowloon'   #This is a nessecity as hong kong does not show up correctly here. 
    if location == 'Mexico City': 
        location = 'IMEXICOC52' # Meixco city works neither
    search_bar.send_keys(location)  #Type the location in. 
    time.sleep(4)
    search_bar.send_keys(Keys.RETURN) #Pressing enter after a delay will send you to a page regarding that location. 
    view_button = driver.find_element_by_xpath('/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div/div/div/div/form/lib-date-selector/div/input')
    view_button.click()
    view_button.click()  #This activate button needs to be clicked twice
    time.sleep(2)  #The code does not scrape properly without these periods of waiting. 
    current_url = driver.current_url
    while len(current_url) == 36: #This is the length of the search page url. If it is this long we need to wait more to get a url with a location code in it.
        time.sleep(1)
        current_url = driver.current_url
    location_urls.append(current_url)  #Add the url to the list of urls. These are inputted in the exact same order as the locations in the locations list. 
    time.sleep(2)
    driver.get(weather_URL)  #Return to searchpage
driver.quit()


#This next part concerns matching each location with the key we can find in the url. 
# No information found for november 2020


In [ ]:
#Here we are matching the location to the specalised key that appears in the url of each url when searching for that location. 

location_key_match = []
url_position = 0
for location in list_of_locations:  #For all locations in this non duplicating list....
    location_entry = {}
    location_entry['Location Name'] = location  #add the name of the location here.
    location_entry['key'] = location_urls[url_position][43:47]   #43 to 47 is the position in the code where the key appears. Also add this to the dictionary.
    location_key_match.append(location_entry)    #This is a list with all the couplings of location name and key
    url_position += 1  # Move to the next URL



In [ ]:
#We are now creating coupluings for the month and the location. This, alongside the previous list, really helps us when scraping the weather page by giving us direct links.
# This tells us which locations are important for each month. 
month_location_match = []
for months in complete_list:   #For all the month variables in the complete exchange list. 
    all_months_locations = []
    current_month_location_match = {}
    for lists in months[1]['Locations']:  #This collection of for loops finds all the locations correpsonding to this month. 
        for items in lists:
            if items not in all_months_locations: #No duplicates
                all_months_locations.append(items)
    current_month_location_match['Month'] = months[0]   # The zero entry here is the name of the month. 
    current_month_location_match['Locations'] = all_months_locations #This dictonary value is the list of all the locations appearing in this month. 
    month_location_match.append(current_month_location_match) #This is a list of these dictionaries



In [43]:
#reset
Giant_dataset = []

In [44]:
def trying_scraping(current_date_dict, driver):  # This function tries scraping from the locations on the weather website. These are try statements so if a page has no info, no info is added.
    try:
       
      high_temp = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[1]/td[1]')
      current_date_dict['High_Temp'] = high_temp.text
    except:
       pass

    try:
      low_temp = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[2]/td[1]')
      current_date_dict['Low_Temp'] = low_temp.text
    except:
       pass

    try:
        day_average = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[1]/tr[3]/td[1]')
        current_date_dict['Day_Average_Temp'] = day_average.text
    except:
       pass

    try:
        precipitation_number = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]')
        current_date_dict['Precipitation_Number'] = precipitation_number.text
    except:
       pass

    try:
       dew_point = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[3]/tr[1]/td[1]')
       current_date_dict['Dew_point'] = dew_point.text
    except:
       pass

    try:
        dew_high = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[3]/tr[2]/td[1]')
        current_date_dict['Dew_High'] = dew_high.text
    except:
       pass

    try:
        dew_low = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[3]/tr[3]/td[1]')
        current_date_dict['Dew_Low'] = dew_low.text
    except:
       pass
    
    try:
        dew_average = driver.find_element_by_xpath('')
        current_date_dict['Dew_Average'] = dew_average.text
    except:
        pass
    
    try:
         max_wind_speed = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[4]/tr[1]/td[1]')
         current_date_dict['Max_Wind_Speed'] = max_wind_speed.text
    except:
        pass

    try:
         visibility = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[4]/tr[2]/td[1]')
         current_date_dict['Visibility'] = visibility.text
    except:
        pass

    try:
        day_length = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[6]/tr[1]/td[1]')
        current_date_dict['Day_length'] = day_length.text
    except:
        pass


In [45]:
def daily_observations_scraping(current_date_dict, driver):  # This is a fucntion which scrapes from the table in the weather section. 
    try:
       observations_body = driver.find_element_by_xpath('/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody')
       list_of_rows = observations_body.find_elements_by_xpath("*")  # These of all the rows of the table. 
       time_data = []
       for row in list_of_rows:
           time_condition = {} # A dictionary recording data for each time in the day in the table. 
           row_elements = row.find_elements_by_xpath("*")
           column_number = 1 # Start with the leftmost column. 
           for column in row_elements:
               if column_number == 1:
                   time_condition['Time'] = column.text # This column is the time column. 
               if column_number == 5: 
                   time_condition['Wind_Direction'] = column.text #This column is the wind direction column. 
               if column_number == 10:
                   time_condition['Rain_Condition'] = column.text   #This column is the worded rain column. 
               column_number += 1
           time_data.append(time_condition)       #collect all the time data together and add them to the dictionary. 
       current_date_dict['Time_Data'] = time_data
    except:
        pass

    

In [ ]:
#Now we have month location match and location key match we can web scrap the weather information
#standard url = https://www.wunderground.com/history/daily/MMMX/date/2021-8-2
#we are going to go through the months and for all the locations within that month, we are going for each day to each location
#This dictonary tells us what month corresponds to what number 
month_number_match = [{'Month': 'January', 'Number': '1'},
                      {'Month': 'February', 'Number': '2'}, 
                      {'Month': 'March', 'Number': '3'}, 
                      {'Month': 'April', 'Number': '4'}, 
                      {'Month': 'May', 'Number': '5'}, 
                      {'Month': 'June', 'Number': '6'},
                      {'Month': 'July', 'Number': '7'},
                      {'Month': 'August', 'Number': '8'},
                      {'Month': 'September', 'Number': '9'},
                      {'Month': 'October', 'Number': '10'},
                      {'Month': 'November', 'Number': '11'},
                      {'Month': 'December', 'Number': '12'},]
driver = webdriver.Chrome('./chromedriver')
for months in month_location_match: #We are iterating through the days, then the locations, then the months. Months is the outer for loop.
    current_month_dict = {}
    amount_of_days = 28 #default amount
    month_number = 0 # Before we know the number of the month, we set it to zero. 
    if months['Month'][1] in ('July', 'May', 'March', 'January', 'December', 'October','August'):  #These months have 31 days
        amount_of_days = 31
    elif months['Month'][1] in ('June', 'April', 'November', 'September'): #These months have 30 days
        amount_of_days = 30
    elif int(months['Month'][0]) %4 == 0: #A test for whether the year is a leap year or not. This is sufficient logic in the span of dates we are looking at. 
        amount_of_days = 29
    for month_entry in month_number_match:  #We are going through the dictionary here to find the corresponding month number to each month we are looking through.
        if month_entry['Month'] == months['Month'][1]:
            month_number = month_entry['Number']    #We need to do this as the URL uses an numerical input for the month value. 
    current_year = months['Month'][0]
    current_month_dict['Month'] = (months['Month'][0], months['Month'][1]) #These last two enteries correspond to the month of the year and the year. 
    current_month_dict['Data'] = [] #This is an empty list which would contain all scrapped data for this month. Each entry corresponds to a day and a location. 
    for locations in months['Locations']:  #We are finding the key for each location here.
        current_key = ''
        for items in location_key_match:  
            if items['Location Name'] == locations:
                current_key = items['key']    #The key is found.
        for i in range(1, amount_of_days + 1):  #This range means for every day of the month
            time.sleep(3) 
            current_date_dict = {}
            current_date_dict['day_and_location'] = {'day': i,'location': locations}  #Input the independent variables into the dictionary
            desired_url = f"https://www.wunderground.com/history/daily/{current_key}/date/{current_year}-{month_number}-{i}"            
            #This desired url takes us to the webpage we want given the location, year, day and month   
            driver.get(desired_url)
            time.sleep(2)
            trying_scraping(current_date_dict, driver)   #We are using the two previous functions for scraping these pages
            daily_observations_scraping(current_date_dict, driver)
            current_month_dict['Data'].append(current_date_dict)   #Add all this date to the data value of this month
            
            
    Giant_dataset.append(current_month_dict)  #Add the month's data to the giant dataset. 
                            

print('Done')
df_all = pd.DataFrame(Giant_dataset)
df_all.to_csv('all_weather.csv', index = False, sep= ';')
#driver.get('https://www.wunderground.com/history/daily/MMMX/date/2021-8-2')

In [37]:
df_loc_key = pd.DataFrame(location_key_match)  #This is rudimentary saving of the scrapped data 
df_loc_key.to_csv('locationtion_key.csv', index = False, sep= ';')

In [41]:
df_mon_loc = pd.DataFrame(month_location_match) #This is rudimentary saving of the scrapped data 
df_mon_loc.to_csv('month_location.csv', index = False, sep= ';')

In [ ]:


df_all = pd.DataFrame(Giant_dataset) #This is rudimentary saving of the scrapped data 
df_all.to_csv('all_weather.csv', index = False, sep= ';')